# Big Data - Project

## TODO TOPIC

### Dragan Postolovsky
### Wojciech Taisner

TODO

A 1-page summary of litterature review on studies investigating stackoverflow data

TODO

An enumeration of "interesting" investigations directions for the 10-year dataset of stack overflow (from 09/2008 to 09/2018)

### What were the questions?

TODO

A report with analysis results (including visualisation, and statistics) of the 10 years data. These results are the fruit of the investigations projected earlier

In [1]:
#files
base = "stackoverflow.com-"

badges = "Badges"
comments = "Comments"
posthistory = "PostHistory"
postlinks = "PostLinks"
posts = "Posts"
tags = "Tags"
users = "Users"
votes = "Votes"

In [2]:
# import dask bag
import dask.bag as db
# import BeatuifulSoup
from bs4 import BeautifulSoup

# read tags bag, select rows only
tags_bag = db.read_text("Tags.tar.xz", compression="xz").str.strip()
tags_bag = tags_bag.filter(lambda x: x.find("row") >= 0)

In [3]:
# non complex parsing (ex badges, tags, etc)
def xml_row_to_tuple(row):
    soup = BeautifulSoup(row, "xml")
    ret = []
    for a, v in soup.row.attrs.items():
        if v.isdigit():
            v = int(v) #integers take less memory :D
        ret.append(v)
    return tuple(ret);
def xml_headers_to_tuple(row):
    soup = BeautifulSoup(row, "xml")
    ret = []
    for a, v in soup.row.attrs.items():
        ret.append(a)
    return tuple(ret);

In [4]:
for row in tags_bag.take(4):
    print(row);
    print(xml_row_to_tuple(row))
    #soup = BeautifulSoup(row, "xml")
    #print(soup.row['Id'], soup.row['TagName'], soup.row['Count'])

<row Id="1" TagName=".net" Count="276857" ExcerptPostId="3624959" WikiPostId="3607476" />
(1, '.net', 276857, 3624959, 3607476)
<row Id="2" TagName="html" Count="789778" ExcerptPostId="3673183" WikiPostId="3673182" />
(2, 'html', 789778, 3673183, 3673182)
<row Id="3" TagName="javascript" Count="1723731" ExcerptPostId="3624960" WikiPostId="3607052" />
(3, 'javascript', 1723731, 3624960, 3607052)
<row Id="4" TagName="css" Count="563816" ExcerptPostId="3644670" WikiPostId="3644669" />
(4, 'css', 563816, 3644670, 3644669)


In [5]:
tags_bag.take(4) # return tuple
test = tags_bag.map(xml_row_to_tuple).compute()
test[0]
#carefull, takes a while

(1, '.net', 276857, 3624959, 3607476)